In [16]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd
import matplotlib.pyplot as plt
import re
import mysql.connector

In [17]:
# Set Up Client Credentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id='541318d201544530a1fa9ac238bd0978',
    client_secret='44fb7acba3a4478a90fc0ae3e1a696ec'))

In [18]:
#My Sql Database Connection

db_config = { 
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'Root',
    'database': 'spotify_db'}

In [19]:
#Connect to the database

connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

In [20]:
# Read Track URLs from file

file_path = "E:\Data_Analytics_training\Portfolio\Portfolio_Projects\Spotify_tracks.txt"
with open(file_path, 'r') as file:
    track_urls = file.readlines()

In [21]:
# Process each URL
for track_url in track_urls:
    track_url = track_url.strip()  # Remove any leading/trailing whitespace
    try:
        # Extract track ID from URL
        track_id = re.search(r'track/([a-zA-Z0-9]+)', track_url).group(1)

        # Fetch track details from Spotify API
        track = sp.track(track_id)

        # Extract metadata
        track_data = {
            'Track Name': track['name'],
            'Artist': track['artists'][0]['name'],
            'Album': track['album']['name'],
            'Popularity': track['popularity'],
            'Duration (minutes)': track['duration_ms'] / 60000
        }

        # Insert data into MySQL
        insert_query = """
        INSERT INTO spotify_tracks (track_name, artist, album, popularity, duration_minutes)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            track_data['Track Name'],
            track_data['Artist'],
            track_data['Album'],
            track_data['Popularity'],
            track_data['Duration (minutes)']
        ))
        connection.commit()

        print(f"Inserted: {track_data['Track Name']} by {track_data['Artist']}")

    except Exception as e:
        print(f"Error processing URL: {track_url}, Error: {e}")

Inserted: Mr. Brightside by The Killers
Inserted: Time is Running Out by Muse
Inserted: Uprising by Muse
Inserted: Despacito by Luis Fonsi
Inserted: Mr. Brightside by The Killers
Inserted: Lover by Taylor Swift
Inserted: Shape of You by Ed Sheeran
Inserted: Every Breath You Take by The Police
Inserted: Prelude for Piano No. 11 in F-Sharp Minor by Eduard Abramyan


In [22]:
# Close the connection
cursor.close()
connection.close()

In [23]:
print("All tracks have been processed and inserted into the database.")

All tracks have been processed and inserted into the database.
